In [8]:
# read data
data = [(m.captures[1][1], m.captures[2][1]) for m in 
        [match(r"Step (\w).*step (\w) can", L) for L in readlines("input7")]];

In [9]:
# build dependency lookup
function build_deps(data)
    v = Dict()
    for (p, q) ∈ data
        if haskey(v, q)
            v[q] = [v[q]..., p]
        else
            v[q] = [p]
        end
    end
    v
end
deps = build_deps(data)

Dict{Any,Any} with 23 entries:
  'E' => ['W', 'G']
  'Z' => ['G']
  'X' => ['Z', 'E', 'S', 'G', 'T']
  'C' => ['A', 'U', 'B', 'M', 'R']
  'B' => ['W']
  'D' => ['K', 'Z']
  'A' => ['K', 'Y']
  'F' => ['T', 'U', 'R', 'B', 'E']
  'Q' => ['K', 'C', 'Z', 'T', 'X', 'R', 'E']
  'P' => ['B', 'D', 'E']
  'M' => ['R', 'G', 'Y', 'V', 'W']
  'J' => ['Y', 'Q', 'P', 'I', 'M']
  'N' => ['S', 'L', 'Z', 'Q', 'V', 'H', 'C', 'T', 'I', 'K', 'P', 'X', 'D', 'U']
  'O' => ['H', 'N', 'X', 'L', 'Q', 'S', 'J']
  'K' => ['G']
  'H' => ['X', 'E', 'P', 'T', 'J', 'Y', 'L', 'A', 'R', 'Q']
  'I' => ['V', 'S', 'P', 'T', 'Z']
  'S' => ['V', 'B']
  'T' => ['G', 'A', 'Z', 'M']
  'U' => ['D', 'M', 'W', 'K']
  'L' => ['F', 'I', 'J', 'M', 'C', 'S', 'R', 'P']
  'Y' => ['B', 'K']
  'V' => ['G']

In [10]:
# Part 1
function find_answer(deps)
    sequence = []
    all_letters = unique(vcat(
                            collect(Iterators.flatten(values(deps))), 
                            collect(keys(deps))))

    # these ones can start immediately
    starters = setdiff!(Set(all_letters), Set(keys(deps)))  

    # which letters are ok to go next? 
    candidates(deps) = setdiff!(
        Set(k for (k,v) ∈ deps if all(x ∈ sequence for x ∈ v)), 
        Set(sequence))

    # logic to elect the best candidate
    elect(letters) = minimum(letters)
    
    letter = pop!(starters, elect(starters))
    while true
        push!(sequence, letter)
                        
        # keep looking for next letter until we exhaust all letters
        next_candidates = [candidates(deps)..., starters...]
        isempty(next_candidates) && break

        # find next candidate.
        # remove from the starters list once elected
        letter = elect(next_candidates)
        letter ∈ starters && pop!(starters, letter)
    end
    join(sequence)
end
find_answer(deps)

"GKRVWBESYAMZDPTIUCFXQJLHNO"

In [11]:
# Part 2
mutable struct Job
    letter::Char
    started_at::Int
    finished_at::Int
end

mutable struct Worker
    jobs::Vector{Job}
    Worker() = new(Job[])
end

function time_available(worker)
    isempty(worker.jobs) ? 0 : worker.jobs[end].finished_at
end

function completed_jobs(workers, at_time)
    jobs = Job[]
    for worker in workers
        for job in worker.jobs
            job.finished_at <= at_time && push!(jobs, job)
        end
    end
    jobs
end

function in_progress_jobs(workers, at_time)
    jobs = Job[]
    for worker in workers
        for job in worker.jobs
            job.started_at <= at_time < job.finished_at && push!(jobs, job)
        end
    end
    jobs
end

function available_workers(workers, at_time)
    filter(w -> time_available(w) <= at_time, workers)
end

function candidate_letters(workers, at_time, deps, starters)
    completed_letters = Set([j.letter for j in completed_jobs(workers, at_time)])
    in_progress_letters = Set([j.letter for j in in_progress_jobs(workers, at_time)])
    ready_letters = Set([k for (k,v) in deps if all(x in completed_letters for x in v)])
    setdiff!(union!(ready_letters, starters), union!(completed_letters, in_progress_letters))
end
                    
# NOTE: change from 1 to 61 for real
function kick_off!(worker, letter, at_time)
    job = Job(letter, at_time, at_time + 61 + Int(letter) - Int('A'))
    push!(worker.jobs, job)
end

function allocate!(workers, letters, at_time)
    for letter in letters
        free_workers = available_workers(workers, at_time)
        !isempty(free_workers) && kick_off!(free_workers[1], letter, at_time)
    end
    workers
end
                
function run_me(workers, deps)
    ts = 0  # timestamp
    all_letters = unique(vcat(
                            collect(Iterators.flatten(values(deps))), 
                            collect(keys(deps))))
    total_number_of_jobs = length(all_letters)
    starters = setdiff!(Set(all_letters), Set(keys(deps)))  
    while true
        length(completed_jobs(workers, ts)) == total_number_of_jobs && break
        letters = candidate_letters(workers, ts, deps, starters)
        allocate!(workers, letters, ts)
        ts += 1
    end
    println("At $ts second, all jobs are finished")
    workers
end

# unit test
# Step C must be finished before step A can begin.
# Step C must be finished before step F can begin.
# Step A must be finished before step B can begin.
# Step A must be finished before step D can begin.
# Step B must be finished before step E can begin.
# Step D must be finished before step E can begin.
# Step F must be finished before step E can begin.

# ==== quick checks ====
# @show starters = setdiff!(Set('A':'F'), Set(keys(deps)))
# @show available_workers(workers, 0)
# @show time_available(workers[1])
# @show time_available(workers[2])
# @show candidate_letters(workers, 0, deps, starters)
# @show allocate!(workers, ['C'], 0)
# @show candidate_letters(workers, 1, deps, starters)
# @show candidate_letters(workers, 2, deps, starters)
# @show candidate_letters(workers, 3, deps, starters)
# @show candidate_letters(workers, 4, deps, starters)
           
# ==== replicate sample case ====
# @show deps = Dict('A' => ['C'], 'F' => ['C'], 'B' => ['A'], 'D' => ['A'], 'E' => ['B', 'D', 'F'])
# @show workers = [Worker(), Worker()]
# run_me(workers, deps)

# ==== solve problem ====
workers = [Worker(), Worker(), Worker(), Worker(), Worker()]
run_me(workers, deps)

At 903 second, all jobs are finished


5-element Array{Worker,1}:
 Worker(Job[Job('R', 0, 78), Job('E', 83, 148), Job('S', 149, 228), Job('A', 230, 291), Job('T', 303, 383), Job('X', 383, 467), Job('Q', 467, 544), Job('J', 544, 614), Job('L', 614, 686), Job('H', 686, 754), Job('N', 754, 828), Job('O', 828, 903)])
 Worker(Job[Job('W', 0, 83), Job('B', 83, 145), Job('Y', 145, 230), Job('M', 230, 303), Job('U', 303, 384), Job('C', 384, 447)])                                                                                                                        
 Worker(Job[Job('G', 0, 67), Job('K', 67, 138), Job('D', 153, 217), Job('P', 217, 293), Job('I', 383, 452)])                                                                                                                                            
 Worker(Job[Job('Z', 67, 153), Job('F', 384, 450)])                                                                                                                                                                               